In [1]:
# setup:
# 1. streamlit for front end
# 2. ollama as LLM
# 3. Langchain embeddings
# 4. ElasticSearch for RAG search

import streamlit as st
from openai import OpenAI
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import requests

# Load env var
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# define client connection
client = OpenAI(api_key=openai_api_key)

# define es client
es_client = Elasticsearch('http://localhost:9200')
print(es_client.info())


c:\Users\hohih\llm\llm\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'name': '870c33757f90', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'HnwsLUN3RJOT7ChvDz3Uyw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [ ]:

# define elast search
def elastic_search(query, index_name = "course-questions"):
    search_query = {
        'size': 5,
        'query': {
            'bool': {
                'must': {
                    'multi_match': {
                        'query': query,
                        'fields': ['question^3', 'text', 'section'],
                        'type': 'best_fields'
                    }
                },
                'filter': {
                    'term': {
                        'course': 'data-engineering-zoomcamp'
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body= search_query)
    result_docs = [hit['_source'] for hit in response['hits']['hits']]
    return result_docs

# create a prompt from template
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

# define llm call
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

# one functon call for rag
def rag(query):
    search_results = elastic_search(query)
    print(search_results)
    prompt = build_prompt(query, search_results)
    #print()
    #print(prompt)
    answer = 'hi'
    #answer = llm(prompt)
    return answer



In [3]:
rag('The course has already started, can I still join it?')

[]

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: The course has already started, can I still join it?

CONTEXT:


'hi'